In [6]:
import sys
print(sys.version)
print(sys.version_info)

3.11.10 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:26) [MSC v.1929 64 bit (AMD64)]
sys.version_info(major=3, minor=11, micro=10, releaselevel='final', serial=0)


In [8]:
import sys
print(sys.executable)


C:\Users\nikde\anaconda3\python.exe


In [10]:
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm

                                              0.0/12.2 MB ? eta -:--:--
                                              0.0/12.2 MB ? eta -:--:--
                                             0.0/12.2 MB 262.6 kB/s eta 0:00:47
                                             0.1/12.2 MB 409.6 kB/s eta 0:00:30
                                             0.1/12.2 MB 654.9 kB/s eta 0:00:19
                                              0.3/12.2 MB 1.1 MB/s eta 0:00:11
     -                                        0.4/12.2 MB 1.5 MB/s eta 0:00:08
     --                                       0.7/12.2 MB 2.1 MB/s eta 0:00:06
     ---                                      1.0/12.2 MB 2.9 MB/s eta 0:00:04
     ----                                     1.4/12.2 MB 3.5 MB/s eta 0:00:04
     -----                                    1.8/12.2 MB 4.0 MB/s eta 0:00:03
     -------                                  2.3/12.2 MB 4.5 MB/s eta 0:00:03
     --------                                 2.5/12.2 MB 4.6 M

In [12]:
import sqlite3
import spacy

nlp = spacy.load("en_core_web_sm")

In [14]:


def get_sql_query(user_input):
    """Converts user query into an SQL query."""
    doc = nlp(user_input.lower())

    # Identify department names
    department_names = ["sales", "engineering", "marketing"]
    department = next((token.text.capitalize() for token in doc if token.text.lower() in department_names), None)

    # Identify date
    date = next((token.text for token in doc if token.ent_type_ == "DATE"), None)

    # Identify query type
    if "employees" in user_input and department:
        return f"SELECT * FROM Employees WHERE Department = '{department}';"
    
    if "manager" in user_input and department:
        return f"SELECT Manager FROM Departments WHERE Name = '{department}';"
    
    if "hired after" in user_input and date:
        return f"SELECT * FROM Employees WHERE Hire_Date > '{date}';"
    
    if "total salary expense" in user_input and department:
        return f"SELECT SUM(Salary) FROM Employees WHERE Department = '{department}';"

    return None

def execute_query(sql):
    """Executes SQL query and fetches results."""
    conn = sqlite3.connect("company.db")
    cursor = conn.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    conn.close()
    return result
